In [245]:
import requests
import matplotlib.pyplot as plt
# from dataprep.eda import *
import pandas as pd
import time 
import numpy as np
import sqlite3

In [246]:
conn = sqlite3.connect('data/crypto.db') # ~代表路徑
c = conn.cursor()

In [247]:
def datetime_to_timestamp(t0):
    t0 = [str(n) for n in t0]
    t = t0[0]+'-'+t0[1]+'-'+t0[2]+' '+t0[3]+':'+t0[4]+':'+'00'
    t = time.strptime(t, "%Y-%m-%d %H:%M:%S")
    t = int(time.mktime(t))
    t *= 1000 # to mili sec
    return t

In [248]:
def download_from_binance(t0,coin='BTCUSDT'):
    # t0 = [str(n) for n in t0]
    # time format : "2022-07-01 00:00:00"
    # time_start = t0[0]+'-'+t0[1]+'-'+t0[2]+' '+t0[3]+':'+t0[4]+':'+'00'
    # time_start = time.strptime(time_start, "%Y-%m-%d %H:%M:%S")
    # time_start = int(time.mktime(time_start))
    # time_start *= 1000 # to mili sec

    arug = {'symbol':coin, 'interval':'1h','startTime':t0,'limit':1000}
    resp = requests.get('https://api.binance.com/api/v3/klines', params=arug)
    # print(resp.status_code)
    data = resp.json()
    df = pd.DataFrame(data).astype(float)
    return df

In [249]:
def save_to_sqlite(conn, c, coin, df):
    s, e = 0, 0
    for _, row in df.iterrows():
        try:
            c.execute('INSERT INTO crypto VALUES '+str((coin, row[0], row[1], row[4], row[5])))
            s += 1
        except sqlite3.Error as err:
            c.execute('UPDATE crypto SET open={}, close={}, volume={} WHERE name="{}" AND time={}'.format(row[1], row[4], row[5], coin, row[0]))
            e += 1
    # print('ADD',s,'UPDATE',e)
    conn.commit()
    return (s,e)

In [250]:
def loop_update_price(coin, start_time):
    start_time = datetime_to_timestamp(start_time)
    request_count = 0
    st, et = 0, 0
    while True:
        df = download_from_binance(start_time,coin)
        if len(df)==0:
            break
        (s,e) = save_to_sqlite(conn, c, coin, df)
        start_time = int(df.iloc[-1].values[0])+1
        st += s
        et += e
        request_count += 1
    return (st, et, request_count)
    

In [251]:
coin = 'ETHUSDT'
start_time = (2000,1,1,0,0)
(s,e,request_count) = loop_update_price(coin, start_time)
print('ADD',s,'UPDATE',e,'with requests :',request_count)

ADD 46849 UPDATE 0 with requests : 47
